In [ ]:
import plotly.io as pio
pio.renderers.default='notebook'
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
from Bio import SeqIO
from collections import OrderedDict
import statsmodels.api as sm
import os

In [ ]:
# inputs
pipeline = "<PIPELINE>"
conf = "<CONF>"
pav_tsv = "<PAV_TSV>"
stepwise_file = "<SYEPWISE_TSV>"
ref_name = "<REF_NAME>"
proteins_fasta = "<PROT_FASTA>"
assembly_stats_tsv = "<STATS_TSV>"

# Pan genome report
**note**: all plots are interactive - you can pan, zoom, hover to get additional information and change the display by clicking legend items.

In [ ]:
now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print("Report generated with pipeline: %s" % pipeline)
print("Using configuration: %s" % conf)
print("Report created %s" % now)
print("Inputs:")
print("PAV tsv: %s" % pav_tsv)
print("Stepwise data: %s" % stepwise_file)
print("Reference name: %s" % ref_name)
print("Pan proteome fasta: %s" % proteins_fasta)
print("Assembly stats tsv: %s" % assembly_stats_tsv)

## General stats

In [ ]:
pav_df = pd.read_csv(pav_tsv, sep='\t', index_col=0)
n_samples = pav_df.shape[1]
n_genes = pav_df.shape[0]

In [ ]:
occupancy_df = pd.DataFrame(pav_df.sum(axis=1))
occupancy_df.columns = ["occupancy"]
occupancy_df = occupancy_df.loc[occupancy_df["occupancy"] > 0]
def category_by_occupancy(occup, core):
    if occup == core:
        return "Core"
    elif occup > 1:
        return "Shell"
    else:
        return "Singleton"

occupancy_df["category"] = occupancy_df["occupancy"].map( lambda x: category_by_occupancy(x, n_samples))

In [ ]:
occup_cat = occupancy_df.groupby("category").count()
occup_cat.columns = ["genes"]

In [ ]:
ref_or_not = pd.DataFrame(pav_df[ref_name].map({1: "Reference", 0: "Non-reference"}))
ref_or_not.columns = ["Type"]
ref_or_not_cat = pd.DataFrame(ref_or_not["Type"].value_counts())
ref_or_not_cat.columns = ["genes"]

In [ ]:
print("Samples in pan-genome: %s" % n_samples)
print("Pan-genes in pan-genome: %s" % n_genes)
core = occup_cat['genes']['Core']
shell = occup_cat['genes']['Shell']
sing = occup_cat['genes']['Singleton']
print("Core pan-genes: %s (%s%%)" %(core, core/n_genes*100))
print("Shell pan-genes: %s (%s%%)" %(shell, shell/n_genes*100))
print("Singleton pan-genes: %s (%s%%)" %(sing, sing/n_genes*100))
ref = ref_or_not_cat['genes']['Reference']
non_ref = ref_or_not_cat['genes']['Non-reference']
print("Reference pan-genes: %s (%s%%)" %(ref, ref/n_genes*100))
print("Non-reference pan-genes: %s (%s%%)" %(non_ref, non_ref/n_genes*100))

## Stepwise sample addition analysis
Samples are added one by one and the number of total genes, core genes and singletons is calculated at each step. The process is repeated 100 times with randomly-chosen sample orders. Lines represent the mean number of genes at each step across the 100 orders. Bands around the lines represent min and max number of genes across the 100 orders.

In [ ]:
stepwise_df = pd.read_csv(stepwise_file, sep='\t')

In [ ]:
stepwise_mean = stepwise_df.groupby(["n_samples", "stat"]).mean()
stepwise_min = stepwise_df.groupby(["n_samples", "stat"]).min()
stepwise_max = stepwise_df.groupby(["n_samples", "stat"]).max()

In [ ]:
n_all_samples = max(stepwise_df["n_samples"])
x = list(range(1,n_all_samples+1))
x_rev = x[::-1]

core_max = list(stepwise_max.xs("Core genes", level="stat")["n_genes"])
core_min = list(stepwise_min.xs("Core genes", level="stat")["n_genes"])
core_min = core_min[::-1]
core_mean = list(stepwise_mean.xs("Core genes", level="stat")["n_genes"])

tot_max = list(stepwise_max.xs("Total genes", level="stat")["n_genes"])
tot_min = list(stepwise_min.xs("Total genes", level="stat")["n_genes"])
tot_min = tot_min[::-1]
tot_mean = list(stepwise_mean.xs("Total genes", level="stat")["n_genes"])

sing_max = list(stepwise_max.xs("Singletons", level="stat")["n_genes"])
sing_min = list(stepwise_min.xs("Singletons", level="stat")["n_genes"])
sing_min = sing_min[::-1]
sing_mean = list(stepwise_mean.xs("Singletons", level="stat")["n_genes"])
sing_min[-1] = np.nan
sing_max[0] = np.nan
sing_mean[0] = np.nan

fig = go.Figure(layout_title_text="Stepwise sample addition",
               layout_xaxis_title="# samples",
               layout_yaxis_title="# genes")

fig.add_trace(go.Scatter(
    x=x+x_rev,
    y=core_max+core_min,
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=True,
    name='Core',
))
fig.add_trace(go.Scatter(
    x=x, y=core_mean,
    line_color='rgb(0,100,80)',
    name='Core',
))

fig.add_trace(go.Scatter(
    x=x+x_rev,
    y=tot_max+tot_min,
    fill='toself',
    fillcolor='rgba(0,176,246,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=True,
    name='Total',
))
fig.add_trace(go.Scatter(
    x=x, y=tot_mean,
    line_color='rgb(0,176,246)',
    name='Total',
))

fig.add_trace(go.Scatter(
    x=x+x_rev,
    y=sing_max+sing_min,
    fill='toself',
    fillcolor='rgba(231,107,243,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=True,
    name='Singletons',
))
fig.add_trace(go.Scatter(
    x=x, y=sing_mean,
    line_color='rgb(231,107,243)',
    name='Singletons',
))


fig.show("notebook")

## Pan genome gene occupancy
The occupancy of a gene is defined as the number of samples in which it is present. A gene with "full" occupancy (i.e. present in all pan genome samples) is called a core gene, while a gene present in only one sample is called a singleton. All other genes are called shell genes.

In [ ]:
fig = px.histogram(occupancy_df, x="occupancy", color="category", nbins=n_samples)
fig.update_layout(title_text="Histogram of pan genome occupancy", yaxis_title="# genes")
fig.show()

## Pan genome composition
The overall composition of the pan genome is shown either by occupancy category or by reference cs. non-reference genes.

In [ ]:
fig = px.pie(occup_cat, names=occup_cat.index, values="genes", title="Pan genome composition - occupancy categories")
fig.show()

In [ ]:
fig = px.pie(ref_or_not_cat, names=ref_or_not_cat.index, values="genes", title="Pan genome composition - reference vs. non-reference")
fig.show()

## Per-sample composition
The plot below shows the composition of each sample in the pan genome in terms of occupancy categories and reference/non-reference.

In [ ]:
def gene_cat(col):
    present = pd.DataFrame(col[col == 1])
    present = present.join(occupancy_df).join(ref_or_not)
    occup_cat = present.groupby("category").count()
    core = 0; shell = 0; singletons = 0
    if "Core" in occup_cat.index:
        core = occup_cat.loc["Core"]["occupancy"]
    if "Shell" in occup_cat.index:
        shell = occup_cat.loc["Shell"]["occupancy"]
    if "Singleton" in occup_cat.index:
        singletons = occup_cat.loc["Singleton"]["occupancy"]
    type_cat = present.groupby("Type").count()
    ref = 0; non_ref = 0
    if "Reference" in type_cat.index:
        ref = type_cat.loc["Reference"]["occupancy"]
    if "Non-reference" in type_cat.index:
        non_ref = type_cat.loc["Non-reference"]["occupancy"]   
    return pd.Series([core, shell, singletons, ref, non_ref])
    
per_sample_composition = pav_df.apply(axis=0, func=gene_cat).transpose()
per_sample_composition.columns = ["Core","Shell","Singleton","Reference","Non-reference"]

In [ ]:
fig = go.Figure(
    data = [
        go.Bar(x=per_sample_composition.index,
              y=per_sample_composition["Core"],
              name="Core",
              offsetgroup=0),
        go.Bar(x=per_sample_composition.index,
              y=per_sample_composition["Shell"],
              name="Shell",
              offsetgroup=0,
              base=per_sample_composition["Core"]
              ),
        go.Bar(x=per_sample_composition.index,
              y=per_sample_composition["Singleton"],
              name="Singleton",
              offsetgroup=0,
               base=per_sample_composition["Shell"] + per_sample_composition["Core"]
              ),
        go.Bar(x=per_sample_composition.index,
              y=per_sample_composition["Reference"],
              name="Reference",
              offsetgroup=1
              ),
        go.Bar(x=per_sample_composition.index,
              y=per_sample_composition["Non-reference"],
              name="Non-reference",
              offsetgroup=1,
               base=per_sample_composition["Reference"]
              ),
    ],
    layout=go.Layout(title="Per-sample composition",
                    yaxis_title="# genes")
)
fig.show()

## Pairwise similarity between samples
Each pair of samples in the pan genome are compared in terms of gene presence/absence, and a similarity score is calculated as the number of genes on which the two samples agree (either presence or absence).

In [ ]:
def compare(col):
    tot = len(col)
    sim = []
    for sample in pav_df.columns:
        sim.append((col == pav_df[sample]).sum()/tot)
    return pd.Series(sim)

pairwise_sim = pav_df.apply(axis=0, func=compare)
pairwise_sim.index = pairwise_sim.columns

In [ ]:
g = sns.clustermap(pairwise_sim, annot=True, cmap="YlGnBu")

## Pan proteome stats
Histograms of protein lengths according to occupancy categories

In [ ]:
prot_lens = {rec.id: [len(rec.seq)] for rec in SeqIO.parse(proteins_fasta, 'fasta')}
prot_lens_df = pd.DataFrame.from_dict(prot_lens, orient='index')
prot_lens_df.columns = ["protein_length"]
prot_lens_df = prot_lens_df.join(occupancy_df, how="inner")

In [ ]:
core = prot_lens_df.loc[prot_lens_df["category"] == "Core"]["protein_length"]
shell = prot_lens_df.loc[prot_lens_df["category"] == "Shell"]["protein_length"]
singleton = prot_lens_df.loc[prot_lens_df["category"] == "Singleton"]["protein_length"]

hist_data = [core, shell, singleton]
group_labels = ["Core", "Shell", "Singleton"]

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, show_rug=False)
fig.update_layout(xaxis_title_text="Protein length", yaxis_title_text="Density", title="Histogram of protein length")
fig.show()

## Assembly stats

In [ ]:
assembly_stats_df = pd.read_csv(assembly_stats_tsv, sep='\t', index_col=0)
assembly_stats_df.index.name = "Sample"
assembly_stats_df["% gaps"] = assembly_stats_df["# N's per 100 kbp"]/1000
display_cols = OrderedDict({"Avg. coverage depth": "Sequecing depth (x)",
                            "Read length (bp)": "Read length (bp)",
                            "Total length": "Assembly size",
                            "# contigs (>= 0 bp)": "# of contigs",
                            "N50": "N50", "N75": "N75",                           
                            "% gaps": "% gaps",
                            "% Complete BUSCOs": "% Complete BUSCOs"})
if "% unmapped (Chr0)" in assembly_stats_df.columns:
    display_cols["% unmapped (Chr0)"] = "% unmapped (Chr0)"
display_cols["QUAST report"] = "QUAST report"
assembly_stats_df = assembly_stats_df[display_cols.keys()]
assembly_stats_df.rename(columns=display_cols, inplace=True)
assembly_stats_df['Read length (bp)'] = assembly_stats_df['Read length (bp)'].apply(str)
def make_clickable(url):
    name= os.path.basename(url)
    return '<a href="{}">{}</a>'.format(url,name)

assembly_stats_df.style.format({'QUAST report': make_clickable})

In [ ]:
fig = px.scatter(assembly_stats_df, x="Sequecing depth (x)", y="Assembly size",
                hover_data=[assembly_stats_df.index,"Sequecing depth (x)","Assembly size"],
                color = "Read length (bp)", title="Sequencing depth vs. assembly size")
regline = sm.OLS(assembly_stats_df['Assembly size'],
                 sm.add_constant(assembly_stats_df['Sequecing depth (x)'])).fit().fittedvalues
fig.add_traces(go.Scatter(x=assembly_stats_df['Sequecing depth (x)'], y=regline, mode = 'lines',
                          marker_color='black', name='Trendline')
                          )
fig.show()

In [ ]:
fig = px.scatter(assembly_stats_df, x="Sequecing depth (x)", y="N50",
                hover_data=[assembly_stats_df.index,"Sequecing depth (x)","N50"],
                color = "Read length (bp)", title="Sequencing depth vs. N50")
regline = sm.OLS(assembly_stats_df['N50'],
                 sm.add_constant(assembly_stats_df['Sequecing depth (x)'])).fit().fittedvalues
fig.add_traces(go.Scatter(x=assembly_stats_df['Sequecing depth (x)'], y=regline, mode = 'lines',
                          marker_color='black', name='Trendline')
                          )
fig.show()